# Local Siderial Time Converter

This calculator will take in an input of the current Julian Day Number and longitude and output the local siderial time (lst). It uses a reference JD of January 1, 1900, so times inputted before this day will raise an error.

In [3]:
from __future__ import division

def to_siderial_time(julian_day_number, longitude, direction):

    # reference data
    reference_julian_number = 2415020.50000
    year = 1900
    month = 1
    day = 1
    hour = 0
    minute = 0
    second = 0

    julian_days_to_first_of_year = 0
    julian_days_to_first_of_previous_year = 0

    vequinox_month = 9
    vequinox_day = 22
    vequinox_hour = 0
    vequinox_minute = 0
    vequinox_second = 0
    vequinox_julian = 0

    siderial_days_in_solar_day = 1.002737904

    # Error checking for reasonable longitude coordinates:
    if longitude < 0 or longitude > 180:
        raise ValueError("Longitude must be between 0 degrees and 180 degrees")

    # Third parameter must specify direction with either "E" or "W"
    if direction != "E" and direction != "W":
        raise ValueError("Direction parameter must be either capital \"E\" or \"W\"")
    
    # For simplicity, program will only count up from start number
    if julian_day_number < reference_julian_number:
        raise ValueError("Julian Day Number must be after %s" % (year - 1))

    julian_difference = julian_day_number - reference_julian_number
    
    days = int(julian_difference) + 1

    # Determine year
    dipy = 0
    while days > 365:
        diy = 365
        leap_year = ((year % 4 == 0) and not (year % 100 == 0)) or (year % 400 == 0)
        if leap_year:
            diy = 366
        if leap_year and days < 366:
            break
        days -= diy
        julian_days_to_first_of_year += diy
        julian_days_to_first_of_previous_year += dipy
        year += 1
        dipy = diy

    # Determine month
    leap_year = ((year % 4 == 0) and not (year % 100 == 0)) or (year % 400 == 0)
    last_year_leap_year = (((year - 1) % 4 == 0) and not ((year - 1) % 100 == 0)) or ((year - 1) % 400 == 0)
    month_durations = {
        1: 31,
        2: (29 if leap_year else 28),
        3: 31,
        4: 30,
        5: 31,
        6: 30,
        7: 31,
        8: 31,
        9: 30,
        10: 31,
        11: 30,
        12: 31
    }

    for month_number in month_durations:
        if days <= month_durations[month_number]:
            month = month_number
            break
        days -= month_durations[month_number]

    # Determine day
    day = days

    # Determine Julian days since last Vernal Equinox
    was_this_year = False
    if month > 9:
        was_this_year = True
    if month == 9 and day >= 22:
        was_this_year = True

    if was_this_year:
        vequinox_julian = reference_julian_number + \
            julian_days_to_first_of_year + \
            (31+29+31+30+31+30+31+31+21 if leap_year else 31+28+31+30+31+30+31+31+21)
    else:
        vequinox_julian = reference_julian_number + \
            julian_days_to_first_of_previous_year + \
            (31+29+31+30+31+30+31+31+21 if last_year_leap_year else 31+28+31+30+31+30+31+31+21)

    days_since_vequinox = julian_day_number - vequinox_julian

    # Convert solar days to siderial days
    siderial_days_since_vequinox = days_since_vequinox * siderial_days_in_solar_day
    siderial_julian_days = vequinox_julian + siderial_days_since_vequinox
    siderial_difference = siderial_julian_days - reference_julian_number

    # Determine hour
    siderial_decimal = siderial_difference - int(siderial_difference)
    hour = int(siderial_decimal * 24)

    # Determine minute
    siderial_decimal = siderial_decimal - hour / 24
    minute = int(siderial_decimal * 24 * 60)

    # Determine second
    siderial_decimal = siderial_decimal - minute / (24 * 60)
    second = int(siderial_decimal * 24 * 60 ** 2)

    # Convert longitude to time correction
    hour_longitude = (longitude / 360) * 24
    
    hour_correction = int(hour_longitude)
    minute_correction = int((hour_longitude - hour_correction) * 60)
    second_correction = int((hour_longitude - hour_correction - minute_correction / 60) * 60 ** 2)

    # Add or subtract time correction based on side of prime meridian
    if direction == "E":
        total_second = second + second_correction
        second_carryover = 0
        if total_second >= 60:
            second = total_second - 60
            second_carryover = 1
        else:
            second = total_second

        total_minute = minute + minute_correction + second_carryover
        minute_carryover = 0
        if total_minute >= 60:
            minute = total_minute - 60
            minute_carryover = 1
        else:
            minute = total_minute

        total_hour = hour + hour_correction + minute_carryover
        if total_hour >= 24:
            hour = total_hour - 24
        else:
            hour = total_hour

    else:
        total_second = second - second_correction
        second_carryover = 0
        if total_second < 0:
            second = total_second + 60
            second_carryover = -1
        else:
            second = total_second

        total_minute = minute - minute_correction + second_carryover
        minute_carryover = 0
        if total_minute < 0:
            minute = total_minute + 60
            minute_carryover = -1
        else:
            minute = total_minute

        total_hour = hour - hour_correction + minute_carryover
        if total_hour < 0:
            hour = total_hour + 24
        else:
            hour = total_hour
        
    return (hour, minute, second)

In [4]:
to_siderial_time(2457581.66667, 72.9279, "W")

(18, 28, 4)